In [1]:
import json
import numpy as np
import sys
sys.path.append('../src')
import cpu

# https://github.com/TomHarte/ProcessorTests/tree/main/nes6502

with open('C:/Users/David/Documents/Coding/daveNES/tests/ProcessorTests-main/nes6502/v1/e9.json') as f:
    test = json.load(f)

pygame 2.3.0 (SDL 2.24.2, Python 3.10.8)
Hello from the pygame community. https://www.pygame.org/contribute.html


`! pytest --html="report_%date:/=-%_%time::=-%.html"`

In [2]:
subtest = 17

In [3]:
test[0]

{'name': 'e9 20 ae',
 'initial': {'pc': 60024,
  's': 171,
  'a': 50,
  'x': 168,
  'y': 179,
  'p': 106,
  'ram': [[60024, 233], [60025, 32], [60026, 174]]},
 'final': {'pc': 60026,
  's': 171,
  'a': 17,
  'x': 168,
  'y': 179,
  'p': 41,
  'ram': [[60024, 233], [60025, 32], [60026, 174]]},
 'cycles': [[60024, 233, 'read'], [60025, 32, 'read']]}

In [4]:
test[subtest]['initial']['ram']

[[46693, 233], [46694, 153], [46695, 0]]

In [5]:
def init_daveNES(test):
    daveNES = cpu.MOS6502(debug = False)
    daveNES.initialise_RAM()

    # Load test program
    for val in test['initial']['ram']:
        print(f'{val[0]}, {val[1]}')
        daveNES.ram.memory[val[0]] = np.uint8(val[1])
    daveNES.r_program_counter = test['initial']['pc']
    daveNES.r_stack_pointer = test['initial']['s']
    daveNES.r_accumulator = test['initial']['a']
    daveNES.r_index_X = test['initial']['x']
    daveNES.r_index_Y = test['initial']['y']
    daveNES.value_to_status(test['initial']['p'])

    return daveNES

In [6]:
daveNES = init_daveNES(test[subtest])

46693, 233
46694, 153
46695, 0


In [7]:
daveNES.step_program()
# daveNES.r_status['flag_B1'] = 1

0xe9, SBC
PC: 0xb665, SP: 0x8c, A: 0x66, X: 0xa9, Y: 0x97, [1, 1, 1, 0, 1, 0, 0, 0]
SBC performed
value =153
a =102
b =153
result =204


In [8]:
print(f'{test[subtest]["final"]["pc"]} : {daveNES.r_program_counter}')
print(f'{test[subtest]["final"]["s"]} : {daveNES.r_stack_pointer}')
print(f'{test[subtest]["final"]["a"]} : {daveNES.r_accumulator}')
print(f'{test[subtest]["final"]["x"]} : {daveNES.r_index_X}')
print(f'{test[subtest]["final"]["y"]} : {daveNES.r_index_Y}')
print(f'{test[subtest]["final"]["p"]:08b} : {"".join(str(int(daveNES.r_status[k])) for k in daveNES.r_status.keys())[::-1]}')

46695 : 46695
140 : 140
204 : 204
169 : 169
151 : 151
11101000 : 11101000


In [9]:
daveNES.r_status['flag_B1']

True

In [10]:
a = daveNES.r_accumulator
value = 53
print(a)
b = 53
result = 340

204


In [11]:
(~(value ^ np.uint8(result)) & (137 ^ np.uint8(result))) & 0x80

128

(data ^ result) & (result ^ self.register_a) & 0x80 != 0